In [1]:
import pandas as pd
import numpy as np

In [14]:
API_KEY = ''
EMAIL_PWD = ''

In [ ]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head()

In [ ]:
import smtplib as s
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import google.generativeai as genai

def generate_email_body(name, position, keywords, company):
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    prompt = f"Write a professional email body for a cold outreach to a recruiter named {name} for the position of {position} at {company}, showing keen interest, relevant skills from given skills, mentioning previous startup experience and ability to build from scratch.\n Skills: {keywords}. Limit the response to 200 words. Replace [Your Name] with Vedant Pople. Don't use any [Company Name]. Don't use any other [ ]."
    response = model.generate_content(prompt)
    return response.text

object = s.SMTP('smtp.gmail.com', 587)
object.ehlo()
object.starttls()
object.login('vpople4@gmail.com', EMAIL_PWD)

for index, row in data.iterrows():
    name = row['Name']
    email = row['Email']
    position = row['Position']
    company = row['Company']
    keywords = row['Key Words']

    subject = f"Excited to apply for {position} position at your company."

    body = generate_email_body(name, position, keywords, company)
    print("Generated email body:")
    print(body)

    user_input = input("Would you like to make changes to the email body? (yes/no): ").lower()

    if user_input == 'yes':
        print("Please enter the modified email body. Press Enter twice when you're done:")
        modified_body = []
        while True:
            line = input()
            if line == "":
                break
            modified_body.append(line)
        body = "\n".join(modified_body)

    msg = MIMEMultipart()
    msg["From"] = 'vpople4@gmail.com'
    msg["To"] = email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, 'plain'))

    filepath2 = r'VM-Pople-CV.pdf'
    attachments = [filepath2]

    for filepath in attachments:
        filename = filepath.split('\\')[-1]
        with open(filepath, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={filename}')
            msg.attach(part)

    send_confirmation = input("Do you want to send this email? (yes/no): ").lower()

    if send_confirmation == 'yes':
        # Send email
        object.send_message(msg)
        print("Email sent successfully!")
    else:
        print("Email sending cancelled.")

object.quit()